In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

path = "./Dataset"  # 데이터셋이 저장된 경로
file_list = os.listdir(path)    # 데이터셋의 목록 읽어오기

actions = []    # 파일 확장자 빼고 제스처 이름만 저장
for i in range(len(file_list)) :
    actions.append(file_list[i][:-4])

seq_length = 10

In [2]:
# 제스처별로 전처리하고 훈련데이터 시험데이터 생성하기
datax, datay = [], []

# for i in range(1):
for i in range(len(file_list)):

    # 데이터프레임 읽고 변수, 라벨 분리
    df = pd.read_csv(f'{path}/{file_list[i]}', header=None)
    dfx = df.iloc[:, :-1].to_numpy()
    dfy = df.iloc[:, -1].to_numpy()

    # 시퀀스길이에 따라 시퀀스데이터로 변환
    for i in range(len(df) - seq_length):
        datax.append(dfx[i: i + seq_length])
        datay.append(dfy[i + seq_length])

In [3]:
# # 정답지 원-핫 인코딩
# from keras.utils.np_utils import to_categorical
# datay = to_categorical(datay, num_classes=len(actions))

# 문제지 정답지 타입 변환
datax = np.array(datax)
datay = np.array(datay)

In [4]:
# 훈련데이터 시험데이터 분리
xtrain, xtest, ytrain, ytest = train_test_split(datax, datay, test_size=0.1)
print(xtrain.shape, xtest.shape, ytrain.shape, ytest.shape)

(342, 10, 15) (39, 10, 15) (342,) (39,)


In [5]:
from keras.models import Sequential
from keras import layers, callbacks

In [6]:
import time
start_time = time.time()

In [70]:
model = Sequential([
    layers.SimpleRNN(64, input_shape=xtrain.shape[1:], activation='relu'),
    layers.Dense(32, activation='relu'),
    layers.Dense(len(actions), activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

validation_split = 0.2
epochs = 20

checkpoint = callbacks.ModelCheckpoint('./Models/model.h5', save_best_only=True)
earlystopping = callbacks.EarlyStopping(patience=10, restore_best_weights=True)

hist = model.fit(
                    xtrain, ytrain,
                    validation_split = validation_split,
                    epochs = epochs,
                    callbacks = [checkpoint, earlystopping],
                    verbose=0
                )

score = model.evaluate(xtest, ytest)

2/2 [==============================] - 0s 2ms/step - loss: 0.0069 - accuracy: 1.0000
